In [ ]:
import datetime
import pygetm.legacy
import pygetm.input
import cartopy.crs as ccrs

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

northsea_path = '../../../getm-setups/NorthSea/Topo/NS6nm.v01.nc'  # path to legacy GETM North Sea test case
era_path = '../../../igotm/data/ERA-interim/20??.nc'               # path to ERA NetCDF file with t2m variable

map_proj = ccrs.PlateCarree()                                      # map projection to use for plotting

In [ ]:
# Load North Sea domain - we'll interpolate to it later
domain = pygetm.legacy.domain_from_topo(northsea_path, nlev=1)

# Obtain temperature variable - thgis should not read the data yet
t2m = pygetm.input.NetCDFVariable.get(era_path, 't2m', preprocess=lambda ds: ds.isel(time=slice(4, -4)))

In [ ]:
# Plot first time of air temperature - full domain
fig, ax = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj})
t2m0 = pygetm.input.Variable(t2m.x[...,0])
t2m0.x.plot()
ax.coastlines()

In [ ]:
# Limit domain to the North Sea region
fig, ax = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj})
t2m0_clip = pygetm.input.LimitRegion(t2m0, domain.lon.min(), domain.lon.max(), domain.lat.min(), domain.lat.max(), periodic_lon=True)
t2m0_clip.x.plot()
ax.coastlines()

In [ ]:

# Interpolate to actual North Sea grid
fig, ax = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj})
t2m0_sip = pygetm.input.SpatialInterpolation(t2m0_clip, domain.lon[1::2,1::2], domain.lat[1::2,1::2])
t2m0_sip.x.plot(x='longitude_', y='latitude_')
ax.coastlines()

In [ ]:
# Test temporal interpolation (full domain)
fig, ax = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj})
t2m_tip = pygetm.input.TemporalInterpolation.get(t2m)
t2m_tip.update(datetime.datetime(2016,5,1))
t2m_tip.x.plot()
ax.coastlines()

In [ ]:
# Full treatment of air temperature as in model:
# - open NetCDF
# - determine extents of region of interest
# - interpolate temporally
# - interpolate spatially
t2m_clip = pygetm.input.LimitRegion(t2m, domain.lon.min(), domain.lon.max(), domain.lat.min(), domain.lat.max(), periodic_lon=True)
t2m_clip_tip = pygetm.input.TemporalInterpolation.get(t2m_clip)
t2m_clip_tip_sip = pygetm.input.SpatialInterpolation(t2m_clip_tip, domain.lon[1::2,1::2], domain.lat[1::2,1::2])

# Set up plot of initial field - we'll animate it later
fig, ax = matplotlib.pyplot.subplots(subplot_kw={'projection': map_proj})
start = datetime.datetime(2016,5,1)
t2m_clip_tip_sip.update(start)
pc = t2m_clip_tip_sip.x.plot(x='longitude_', y='latitude_', vmin=273.15 + 5, vmax=273.15 + 25)
ax.coastlines()

In [ ]:
# Update air temperature in 15 minute steps over a three month-long period
for i in range(3 * 24 * 4 * 30):
    dt = start + datetime.timedelta(seconds=i * 900)
    t2m_clip_tip_sip.update(dt)
    pc.set_array(t2m_clip_tip_sip.x.values.ravel())
    ax.set_title(dt)
    fig.canvas.draw()